In [0]:
%pip install arize-phoenix langchain openai tiktoken
%pip install protobuf==3.19.4

In [0]:
import json
import os
from getpass import getpass
from urllib.request import urlopen

import numpy as np
import openai
import pandas as pd
import phoenix as px
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import KNNRetriever
from phoenix.trace.langchain import LangChainInstrumentor, OpenInferenceTracer
from tqdm import tqdm

In [0]:
px.launch_app()

In [0]:
import os
import openai
from langchain.llms import AzureOpenAI

In [0]:
openai.api_type = "azure" 
openai.api_base =  "https://kf-llm.openai.azure.com/" # Your Azure OpenAI resource's endpoint value.
openai.api_key = "d6e77295870f4e0fb5f44e2b96838801"
os.environ["OPENAI_API_KEY"] = "d6e77295870f4e0fb5f44e2b96838801"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"

In [0]:
import phoenix as px
import pandas as pd
import numpy as np

# Launch phoenix
session = px.launch_app()

# Once you have started a Phoenix server, you can start your LangChain application with the OpenInferenceTracer as a callback. To do this, you will have to instrument your LangChain application with the tracer:

from phoenix.trace.langchain import OpenInferenceTracer, LangChainInstrumentor

# If no exporter is specified, the tracer will export to the locally running Phoenix server
tracer = OpenInferenceTracer()
LangChainInstrumentor(tracer).instrument()

# Initialize your LangChain application
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import KNNRetriever
from langchain.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceEmbeddings

# Load embeddings model
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                   model_kwargs={'device': 'cpu'})
documents_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/langchain-pinecone/database.parquet"
)
knn_retriever = KNNRetriever(
    index=np.stack(documents_df["text_vector"]),
    texts=documents_df["text"].tolist(),
    embeddings=OpenAIEmbeddings(),
)
chain_type = "stuff"  # stuff, refine, map_reduce, and map_rerank
chat_model_name = "gpt-3.5-turbo"
llm = AzureOpenAI(
    deployment_name="kf-gpt",
    model_name="gpt-35-turbo",temperature= 0.3
)
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type=chain_type,
    retriever=knn_retriever,
)

# Instrument the execution of the runs with the tracer. By default the tracer uses an HTTPExporter
query = "What is euclidean distance?"
# response = chain.run(query, callbacks=[tracer])

# By adding the tracer to the callbacks of LangChain, we've created a one-way data connection between your LLM application and Phoenix.

# To view the traces in Phoenix, simply open the UI in your browser.
session.url

In [0]:
from langchain.document_loaders import GitbookLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import VectorDBQA
import openai
from langchain.llms import AzureOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.agents.agent_toolkits import create_vectorstore_agent,VectorStoreToolkit,VectorStoreInfo
from datetime import datetime
from langchain.callbacks import StdOutCallbackHandler
from langchain.callbacks.arize_callback import ArizeCallbackHandler
from langchain.callbacks.base import CallbackManager

In [0]:
# Load Arize doc data from gitbook
loader = GitbookLoader("https://docs.arize.com/",load_all_paths=True)

pages_data = loader.load()